In [1]:
import camelot
import pandas as pd

In [2]:
pd.options.display.max_rows = 5000

In [3]:
def create_dataframe(tables):
    """ 
    Function to clean up and combine the tables retrieved from each page of the pdf file

    :param TableList tables: List of tables retrieved from the PDF
    """
    # Combine all tables in the TableList to one dataframe
    df = pd.concat([table.df for table in tables], ignore_index = True)
    
    # Set first row as column header
    headers = df.iloc[0].values
    df.columns = headers
    df.drop(index = 0, axis = 0, inplace = True)

    # Set new column names
    df.columns = ['number','title','language','certificate_no','certificate_date','certified_duration','type_of_film','producer_name','production_house','certificate_type']

    df.loc[(df['number'] == ''),['number','title']] = df[(df['number'] == '')]['title'].str.split(' ', n = 1, expand = True).rename(columns = {0:'number',1:'title'})
    
    # Remove all rows with section headers, reset the dataframe index and remove number column
    df['number'] = pd.to_numeric(df['number'], errors='coerce')
    df = df[df['number'].notnull()]
    df.drop('number', axis = 1, inplace = True)
    df.reset_index(drop = True, inplace = True)

    return df

In [4]:
def set_columntype(df):
    """ 
    Function to set the correct column data types for certificate_date and certified_duration

    :param DataFrame df: DataFrame on which to apply the column transformations
    """
    # Classify date column by format type
    df['format'] = 1
    df.loc[df['certificate_date'].str.contains('-', na = False), 'format'] = 2

    # Convert to datetime with different format settings
    df.loc[df['format'] == 1, 'new_date'] = pd.to_datetime(df.loc[df['format'] == 1, 'certificate_date'], format='%d %B %Y')#.dt.strftime('%Y-%m-%d')
    df.loc[df['format'] == 2, 'new_date'] = pd.to_datetime(df.loc[df['format'] == 2, 'certificate_date'], format='%d-%m-%Y')#.dt.strftime('%Y-%m-%d')
    #df['new_date'] = pd.to_datetime(df['certificate_date'])

    df[['mins','secs']] =  df['certified_duration'].str.split('.', expand = True)
    df['mins'].fillna(0, inplace = True)
    df['secs'].fillna(0, inplace = True)
    df['duration_in_seconds'] = (df['mins'].astype(int) * 60) + df['secs'].astype(int)
    df['certified_duration'] = pd.to_timedelta(df['duration_in_seconds'], unit = 's')
    df.drop(['mins','secs','certificate_date'], axis = 1, inplace = True)
    df.rename(columns = {'new_date': 'certificate_date'}, inplace = True)

    return df

In [5]:
tables_2021 = camelot.read_pdf('Films_certified_2021.pdf', pages = '1-end', strip_text = '\n', line_scale = 50)

In [6]:
# camelot.plot(tables_2021[0], kind='grid').show()
# camelot.plot(tables_2021[0], kind='contour').show()
# camelot.plot(tables_2021[0], kind='line').show()
# camelot.plot(tables_2021[0], kind='joint').show()

In [7]:
df1 = create_dataframe(tables_2021)

In [10]:
df1 = set_columntype(df1)

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   title                3870 non-null   object         
 1   language             3870 non-null   object         
 2   certificate_no       3870 non-null   object         
 3   certified_duration   3870 non-null   timedelta64[ns]
 4   type_of_film         3870 non-null   object         
 5   producer_name        3870 non-null   object         
 6   production_house     3870 non-null   object         
 7   certificate_type     3870 non-null   object         
 8   format               3870 non-null   int64          
 9   certificate_date     3870 non-null   datetime64[ns] 
 10  duration_in_seconds  3870 non-null   int32          
dtypes: datetime64[ns](1), int32(1), int64(1), object(7), timedelta64[ns](1)
memory usage: 317.6+ KB


In [14]:
df1.to_csv('film_ratings_2021.csv', sep = '|', encoding='utf-8')

In [ ]:
tables_2022 = camelot.read_pdf('Films_certified_2022.pdf', pages = '1-end', strip_text = '\n', line_scale = 40) # pg 1,7,10, 

In [ ]:
df2 = create_dataframe(tables_2022)

In [ ]:
df2['no_and_date'] = df2['certificate_no'] + " " + df2['certificate_date']
df2['no_and_date'] = df2['no_and_date'].str.strip()
df2['duration_and_type'] = df2['certified_duration'] + " " + df2['type_of_film']
df2['duration_and_type'] = df2['duration_and_type'].str.strip()
df2.drop(['certificate_no','certificate_date', 'certified_duration','type_of_film'], axis = 1, inplace = True)
df2[['certificate_no','certificate_date']] = df2['no_and_date'].str.split(' ', n = 1, expand=True)
df2[['certified_duration','type_of_film']] = df2['duration_and_type'].str.split(' ', n = 1, expand=True)
df2 = df2[['title','language','certificate_no','certificate_date','certified_duration','type_of_film','producer_name','production_house','certificate_type']]

In [ ]:
df2.loc[(df2['language']==''),['title','language']] = df2[(df2['language']=='')]['title'].str.rsplit(' ', n = 1, expand = True).rename(columns = {0:'title',1:'language'})

In [ ]:
df2 = set_columntype(df2)

In [ ]:
df2.info()

In [ ]:
df2.to_csv('film_ratings_2022.csv', sep = '|', encoding='utf-8')

In [ ]:
# 2023 certified films part 1 - pages 1 to 8
tables_2023 = camelot.read_pdf('Films_certified_2023.pdf', pages = '1-8', strip_text = '\n', line_scale = 40)

In [ ]:
# 2023 certified films part 2 - page 9 extracted with higher line_scale of 120
tables_2023_1 = camelot.read_pdf('Films_certified_2023.pdf', pages = '9', strip_text = '\n', line_scale = 120)
tables_2023_1[0].df.drop([0,2,6,7,8], axis=1, inplace = True)
tables_2023_1[0].df[9].replace('',np.nan, inplace = True)
tables_2023_1[0].df.dropna(subset=[9], inplace=True)

In [ ]:
# 2023 certified films part 3 - pages 10 to end of file
tables_2023_2 = camelot.read_pdf('Films_certified_2023.pdf', pages = '10-end', strip_text = '\n', line_scale = 40)

In [ ]:
tables_2023_2[0].df.loc[-1] = ['number','title','language','certificate_no','certificate_date','certified_duration','type_of_film','producer_name','production_house','certificate_type']
tables_2023_2[0].df.index = tables_2023_2[0].df.index + 1
tables_2023_2[0].df.sort_index(inplace=True)

In [ ]:
# Combine all 3 parts into 1 data frame
df = create_dataframe(tables_2023)
df3 = pd.concat([df,tables_2023_1[0].df.rename(columns = {1:'title',3:'language',4:'certificate_no',5:'certificate_date',9:'certified_duration',10:'type_of_film',11:'producer_name',12:'production_house',13:'certificate_type'}),create_dataframe(tables_2023_2)], ignore_index = True)

In [ ]:
df3.info()

In [ ]:
df3 = set_columntype(df3)

In [ ]:
df3.to_csv('film_ratings_2023.csv', sep = '|', encoding='utf-8')

In [ ]:
df_final = pd.concat([df1,df2,df3], ignore_index = True)

In [ ]:
df_final.info()